# IID 와 Non-IID
IID(IIndependent and Identically Distributed)는 데이터들이 서로 아무런 관련이 없고, 균일하게 분포되어있는 데이터 분포를 말함
MINIST예제에서는 0-9 까지의 클래스들을 균등하게 랜덤 추출한 서브 데이터셋을 말함

Non-IID(Non-Independent and Identically Distributed) 불균형 데이터 분포
연합 학습 관련 자료를 보면 Non-IID 세팅에 대한 성능 평가가 꼭 나오는데 연합학습에서 각 클라이언트가 보유한 데이터가 IID세팅처럼 이상적이지 않을 것이기 때문임
그래서 Non-IID 세팅은 어떤 클라이언트에는 0 ~ 3 클래스의 데이터만, 4 ~ 7까지의 데이터만을 추출하여 클래스 간 불균형 데이터 분포를 각 클라이언트의 모델에서 학습할 수 있도록 세팅을 하는 것이다. 
그렇기에 Non-IID 세팅이 IID세팅보다 성능이 더 낮을 수 밖에 없음

In [17]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [18]:
def mnistIID(dataset, num_clients):
    images = int(len(dataset)/num_clients) #clients가 10개일경우에는 6000개
    clients_dict, indices = {}, [i for i in range(len(dataset))]
    for i in range(num_clients):
        np.random.seed(i) #랜덤 시드를 고정하여
        #총 60,000개 indices에서 60000/num_clients로 나눠준 수 만큼 랜덤 균등 분포로 추출함
        clients_dict[i] = set(np.random.choice(indices, images, replace=False)) #random.choice함수가 그역할
        indices = list(set(indices) - clients_dict[i]) #선택한 인덱스에 해당하는 데이터를 추출하려는 곳에서 제거함
    return clients_dict

In [19]:
def mnistNonIID(dataset, num_clients, test=False):
    classes, images = 100, 600 #10개의 클라이언트가 10개의 클래스를 선택하면 총 100개의 클래스들을, 600개씩 추출한다.
    if test:
        classes, images = 20, 500    
    classes_idx = [i for i in range(classes)] # 0~99 classes idx 생성
    clients_dict = {i: np.array([]) for i in range(num_clients)} # 클라이언트 수만큼 생성 10개로 가정
    indices = np.arange(classes*images) #60,000개 이미지 순서 
    
    unsorted_labels = dataset.train_labels.numpy() #훈련 데이터의 클래스 순서 그대로 가져오기
    
    if test:
        unsorted_labels = dataset.test_labels.numpy()
    
    indices_unsorted_labels = np.vstack((indices, unsorted_labels)) # 60,000개 이미지와 레이블 번호가 같이 세팅됨
    indices_labels = indices_unsorted_labels[:,indices_unsorted_labels[1,:].argsort()] #두번째 행을 기준으로 정렬
    indices = indices_labels[0,:]
    
    for i in range(num_clients):
        temp = set(np.random.choice(classes_idx, 2, replace=False)) #2개의 추출 기준을 획득
        classes_idx = list(set(classes_idx) - temp) # 추출된 2개 기준을 삭제
        for t in temp:
            clients_dict[i] = np.concatenate(
                (clients_dict[i], indices[t*images:(t+1)*images]), axis=0) #선택된 추출 기준별 600개를 선택해서 추출
    return clients_dict

In [20]:
def mnistNonIIDUnequal(dataset, num_clients, test=False):
    classes, images = 100, 600 #total60k
    if test:
        classes, images = 200, 50
    classes_idx = [i for i in range(classes)]
    clients_dict = {i: np.array([]) for i in range(num_clients)}
    indices = np.arange(classes*images)
    unsorted_labels = dataset.train_labels.numpy()

    indices_unsorted_labels = np.vstack((indices, unsorted_labels))
    indices_labels = indices_unsorted_labels[:, indices_unsorted_labels[1, :].argsort()]
    indices = indices_labels[0, :]

    min_cls_per_client = 1
    max_cls_per_client = 10
    #1~10까지 랜덤하게 선택한 숫자를 차례대로 출력
    random_selected_classes = np.random.randint(min_cls_per_client, max_cls_per_client+1, size=num_clients)
    #해당 수/전체 수를 하여 %를 구하고 거기에 원하는 전체 갯수를 곱하면 얻을 갯수를 구할 수 있음
    random_selected_classes = np.around(random_selected_classes / sum(random_selected_classes) * classes)
    #소수점으로 구해지기때문에 타입 변환
    random_selected_classes = random_selected_classes.astype(int)
    
    #예외 처리 원하는 갯수보다 많이 획득됬을 경우
    if sum(random_selected_classes) > classes:
        for i in range(num_clients):
            np.random.seed(i)
            #적어도 1개의 클래스를 갖도록 추출 1개의 클래스 우선 추출
            temp = set(np.random.choice(classes_indx, 1, replace=False))
            classes_indx = list(set(classes_indx) - temp)
            for t in temp:
                clients_dict[i] = np.concatenate((clients_dict[i], indices[t*images:(t+1)*images]), axis=0)
                
        random_selected_classes = random_selected_classes-1
        
        for i in range(num_clients):
            if len(classes_idx) == 0: #추출해야할 것들을 모두 추출하였음
                continue
            class_size = random_selected_classes[i] #랜덤하게 결정한 classes size가 idx크기보다 크면 크기를 최대값에 맞춰줌
            if class_size > len(classes_idx): 
                class_size = len(classes_idx)
            np.random.seed(i)
            temp = set(np.random.choice(classes_idx, class_size, replace=False))
            classes_indx = list(set(classes_idx) - temp)
            for t in temp:
                users_dict[i] = np.concatenate((clients_dict[i], indices[t*images:(t+1)*images]), axis=0)
    #적게 혹은 같게 획득했을 경우
    else:

        for i in range(num_clients):
            class_size = random_selected_classes[i]
            np.random.seed(i)
            temp = set(np.random.choice(classes_indx, class_size, replace=False))
            classes_indx = list(set(classes_indx) - temp)
            for t in temp:
                clients_dict[i] = np.concatenate((clients_dict[i], indices[t*images:(t+1)*images]), axis=0)

        if len(classes_idx) > 0: #idx에서 아무도 가져가지 않았을때(남아있는 경우)
            class_size = len(classes_indx)
            k = min(clients_dict, key=lambda x: len(clients_dict.get(x)))
            temp = set(np.random.choice(classes_idx, class_size, replace=False))
            classes_idx = list(set(classes_idx) - temp)
            for t in temp:
                clients_dict[k] = np.concatenate((clients_dict[k], indices[t*images:(t+1)*images]), axis=0)

    return clients_dict

In [21]:
def load_dataset(num_clients, iidtype):
    trainset = datasets.MNIST(root='./',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
    testset = datasets.MNIST(root='./',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)
    train_group, test_group = None, None
    if iidtype == 'iid' :
        traingroup = mnistIID(trainset, num_clients)
        testgroup = mnistIID(testset, num_clients)
    elif iidtype == 'noniid':
        traingroup = mnistNonIID(trainset, num_clients)
        testgroup = mnistNonIID(testset, num_clients, True)
    
    return trainset, testset, traingroup, testgroup

In [22]:
class FedDataset(Dataset):
    def __init__(self, dataset, idx):
        self.dataset = dataset
        self.idx = [int(i) for i in idx]
    
    def __len__(self):
        return len(self.idx)
    
    def __getitem__(self, item):
        images, label = self.dataset[self.idx[item]]
        return torch.tensor(images).clone().detach(), torch.tensor(label).clone().detach()

In [23]:
def getImgs(dataset, indices, batch_size):
    return DataLoader(FedDataset(dataset, indices), batch_size=batch_size, shuffle=True)

In [24]:
def getData(dataset, indices):
    return FedDataset(dataset, indices)